# Lista 2 Zuzanna Sosnowska

In [1]:
import scipy
import numpy as np
import copy

In [21]:
def calculate_upper_triangular_matrix(A, b):
    """
    Funkcja, która przekształca zadaną macierz na macierz górnotrójkątną i równolegle przekształca wektor b 
    :param A: Macierz przekształcana do macierzy górnotrójkątnej
    :param b: Równolegle przekształcany wektor b
    :return: Przekształcona macierz A i wektor b
    """
    n = len(A)
    for column in range(n):
        find_nullifying_row = False
        for row in range(column, n):
            if A[row][column] != 0 and find_nullifying_row == False:
                swap_rows(A, row, column)
                swap_elements(b, row, column)
                find_nullifying_row = True
                continue
            if find_nullifying_row:
                a = calculate_nullify_multiplier_for_index(
                    row_to_nullify=A[row],
                    base_row=A[column],
                    index=column
                )
                a = multiply_and_change_rows(A[row], A[column], column)
                change_elements(b, row, column, a)
        if not find_nullifying_row:
            raise ValueError("Macierz osobliwa")


def upper_triangular_matrix(A):
    n = len(A)
    for j in range(n):
        find_nullifying_row = False
        for i in range(j, n):
            if A[i][j] != 0 and find_nullifying_row == False:
                swap_rows(A, i, j)
                find_nullifying_row = True
                continue
            if find_nullifying_row:
                multiply_and_change_rows(A[i], A[j], j)
        if not find_nullifying_row:
            raise ValueError("Macierz osobliwa")


def swap_rows(A, i, j):
    A[i], A[j] = A[j], A[i]


def calculate_nullify_multiplier_for_index(row_to_nullify, base_row, index):
    return row_to_nullify[index] / base_row[index]


def multiply_and_change_rows(row1, row2, j):
    if len(row1) != len(row2):
        raise ValueError("Rows must have same length")
    a = row1[j] / row2[j]
    for i in range(j, len(row1)):
        row1[i] = row1[i] - a * row2[i]
    return a


def change_rows(row1, row2, a):
    if len(row1) != len(row2):
        raise ValueError("Rows must have same length")
    for i in range(len(row1)):
        row1[i] = row1[i] - a * row2[i]


def swap_elements(b, i, j):
    number_copy = b[i]
    b[i] = b[j]
    b[j] = number_copy


def change_elements(b, i, j, a):
    b[i] = b[i] - a * b[j]


def subtract_columns(b, a, j):
    for i in range(j):
        b[i] = b[i] - a[i]


def multiply_columns(a, c, j):
    a_copy = a.copy()
    for i in range(j):
        a_copy[i] = a[i] * c
    return a_copy


def gaussian_solver(A, b):
    A, b = copy.deepcopy(A), copy.deepcopy(b)
    calculate_upper_triangular_matrix(A, b)
    solutions = [0 for _ in range(len(A))]
    solutions[-1] = b[-1] / A[-1][-1]
    for i in range(len(A)-2, -1, -1):
        a_copy =  multiply_columns([A[j][i+1] for j in range(len(A))], solutions[i+1], i+1)
        subtract_columns(b, a_copy, i+1)
        solutions[i] = b[i] / A[i][i]
    return solutions


def det(A):
    A = copy.deepcopy(A)
    upper_triangular_matrix(A)
    det = 1
    for i in range(len(A)):
        det *= A[i][i]
    return det


def multiply_matrix_by_vector(A, b):
    if len(A[0]) != len(b):
        raise ValueError("Matrix must have same length")
    product = [0] * len(A)
    for i in range(len(A)):
        product[i] = multiply_vector_by_vector(A[i], b)
    return product


def multiply_vector_by_vector(a, b):
    if len(a) != len(b):
        raise ValueError("Vector must have same length")
    product = 0
    for i in range(len(a)):
        product += a[i] * b[i]
    return product


def generate_identity_matrix(n):
    return [[1 if i == j  else 0 for i in range(n)] for j in range(n)]


def multiply_row(row, a):
    for i in range(len(row)):
        row[i] = a * row[i]


def inverse_matrix(A):
    A = copy.deepcopy(A)
    n = len(A)
    I = generate_identity_matrix(n)
    for j in range(n):
        c = False
        for i in range(j, n):
            if A[i][j] != 0 and c == False:
                swap_rows(A, i, j)
                swap_rows(I, i, j)
                c = True
                continue
            if c:
                a = multiply_and_change_rows(A[i], A[j], j)
                change_rows(I[i], I[j], a)
        if not c:
            raise ValueError("Macierz osobliwa")

    for j in range(n-1, -1, -1):
        for i in range(j):
            a = multiply_and_change_rows(A[i], A[j], j)
            change_rows(I[i], I[j], a)
        multiply_row(I[j], 1 / A[j][j])
        multiply_row(A[j],1 / A[j][j])
    return I


### Zadanie 4

In [3]:
A = [[0, 0, 2, 1, 2],
     [0, 1, 0, 2, -1],
     [1, 2, 0, -2, 0],
     [0, 0, 0, -1, 1],
     [0, 1, -1, 1, -1]]
b = [1, 1, -4, -2, -1]

print(gaussian_solver(A, b))
print(scipy.linalg.solve(A, b))

[2.0, -2.0, 1.0, 1.0, -1.0]
[ 2. -2.  1.  1. -1.]


### Zadanie 5

In [4]:
def polynomial_coeffs_lst(x0):
    return [x0 ** i for i in range(5)]


points = [(0, -1), (1, 1), (3, 3), (5, 2), (6, -2)]
A = []
b = []
for x, y in points:
    A.append(polynomial_coeffs_lst(x))
    b.append(y)

print(gaussian_solver(A, b))
print(scipy.linalg.solve(A, b))


[-1.0, 2.6833333333333336, -0.8750000000000003, 0.21666666666666679, -0.02500000000000001]
[-1.          2.68333333 -0.875       0.21666667 -0.025     ]


### Zadanie 6

In [28]:
A = [[3.50, 2.77, -0.76, 1.80],
     [-1.80, 2.68, 3.44, -0.09],
     [0.27, 5.07, 6.90, 1.61],
     [1.71, 5.45, 2.68, 1.71]]

b = [7.31, 4.23, 13.85, 11.55]

print(det(A))
x = gaussian_solver(A, b)
print(multiply_matrix_by_vector(A, x))

print('\n')

print(scipy.linalg.det(A, b))
x = scipy.linalg.solve(A, b)
print(A @ x)

-0.22579734000001275
[7.31, 4.230000000000002, 13.849999999999998, 11.550000000000002]


-0.22579733999999005
[ 7.31  4.23 13.85 11.55]


### Zadanie 7

In [29]:
A = [[10, -2, -1, 2, 3, 1, -4, 7],
     [5, 11, 3, 10, -3, 3, 3, -4],
     [7, 12, 1, 5, 3, -12, 2, 3],
     [8, 7, -2, 1, 3, 2, 2, 4],
     [2, -15, -1, 1, 4, -1, 8, 3],
     [4, 2, 9, 1, 12, -1, 4, 1],
     [-1, 4, -7, -1, 1, 1, -1, -3],
     [-1, 3, 4, 1, 3, -4, 7, 6]]

b = [0, 12, -5, 3, -25, -26, 9, -7]

print(gaussian_solver(A, b))
print(scipy.linalg.solve(A, b))

[-1.  1. -1.  1. -1.  1. -1.  1.]
[-1.  1. -1.  1. -1.  1. -1.  1.]


### Zadanie 8

In [12]:
A = [[2, -1, 0, 0, 0, 0],
     [-1, 2, -1, 0, 0, 0],
     [0, -1, 2, -1, 0, 0],
     [0, 0, -1, 2, -1, 0],
     [0, 0, 0, -1, 2, -1],
     [0, 0, 0, 0, -1, 5]]

print(np.array(inverse_matrix(A)), '\n')
print(scipy.linalg.inv(A))

[[0.84 0.68 0.52 0.36 0.2  0.04]
 [0.68 1.36 1.04 0.72 0.4  0.08]
 [0.52 1.04 1.56 1.08 0.6  0.12]
 [0.36 0.72 1.08 1.44 0.8  0.16]
 [0.2  0.4  0.6  0.8  1.   0.2 ]
 [0.04 0.08 0.12 0.16 0.2  0.24]] 

[[0.84 0.68 0.52 0.36 0.2  0.04]
 [0.68 1.36 1.04 0.72 0.4  0.08]
 [0.52 1.04 1.56 1.08 0.6  0.12]
 [0.36 0.72 1.08 1.44 0.8  0.16]
 [0.2  0.4  0.6  0.8  1.   0.2 ]
 [0.04 0.08 0.12 0.16 0.2  0.24]]


### Zadanie 9

In [32]:
A = [[1, 3, -9, 6, 4],
     [2, -1, 6, 7, 1],
     [3, 2, -3, 15, 5],
     [8, -1, 1, 4, 2],
     [11, 1, -2, 18, 7]]


print(np.array(inverse_matrix(A)))
print(scipy.linalg.inv(A), '\n')
print(np.linalg.inv(A))

[[-4.63853894e+13 -4.63853894e+13 -4.63853894e+13 -9.27707788e+13
   9.27707788e+13]
 [-2.20330600e+14 -2.20330600e+14 -2.20330600e+14 -4.40661199e+14
   4.40661199e+14]
 [-3.68974689e+13 -3.68974689e+13 -3.68974689e+13 -7.37949377e+13
   7.37949377e+13]
 [-5.00000000e-01 -5.00000000e-01  5.00000000e-01  0.00000000e+00
   0.00000000e+00]
 [ 9.38249922e+13  9.38249922e+13  9.38249922e+13  1.87649984e+14
  -1.87649984e+14]]
[[-2.22649869e+15 -2.22649869e+15 -2.22649869e+15 -4.45299738e+15
   4.45299738e+15]
 [-1.05758688e+16 -1.05758688e+16 -1.05758688e+16 -2.11517376e+16
   2.11517376e+16]
 [-1.77107851e+15 -1.77107851e+15 -1.77107851e+15 -3.54215701e+15
   3.54215701e+15]
 [-1.11022302e-16 -0.00000000e+00  1.00000000e+00  1.00000000e+00
  -1.00000000e+00]
 [ 4.50359963e+15  4.50359963e+15  4.50359963e+15  9.00719925e+15
  -9.00719925e+15]] 

[[-2.22649869e+15 -2.22649869e+15 -2.22649869e+15 -4.45299738e+15
   4.45299738e+15]
 [-1.05758688e+16 -1.05758688e+16 -1.05758688e+16 -2.11517376